In [1]:
import subprocess
import json
import os
import bit

from constants import *

from bit import wif_to_key
from web3 import Web3
from eth_account import Account
from web3.middleware import geth_poa_middleware

In [2]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [3]:
mnemonic = os.getenv('MNEMONIC', 'meadow wreck estate tell huge letter sure unable one hurdle six gaze')
mnemonic

'meadow wreck estate tell huge letter sure unable one hurdle six gaze'

In [4]:
def derive_wallets(password, coin):
    
    command = f'php hd-wallet-derive.php -g --mnemonic="{password}" --cols=path,address,privkey,pubkey --coin={coin} --numderive=3 --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output,err = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    
    return keys

In [5]:
coins = {}
coins["eth"] = derive_wallets(mnemonic, ETH)
coins["btc-test"] = derive_wallets(mnemonic, BTCTEST)

In [6]:
coins

{'eth': [{'path': "m/44'/60'/0'/0/0",
   'address': '0x450855436748256107d1F8e3F772c15cD8A3B2C9',
   'privkey': '0x4f663489c0da39ddf6a9c9d0a6e12439597b6b73d44a287a4191f32c0af14522',
   'pubkey': '0259bf190395eb0a807be4c0b8a75b743ec7ac7d669b33ee2a5c06e354e5e7085e'},
  {'path': "m/44'/60'/0'/0/1",
   'address': '0x0F90890C0185Eb90Ed87445224bEE237fd463EA0',
   'privkey': '0x7d2892aa4c9afd2bd462d836c2a3ecef749aa9ecdeaca467d39435300b87cfab',
   'pubkey': '029c0f2167349b2bb303d257ccf4e5866c89cd58f872615c845c757998278da81c'},
  {'path': "m/44'/60'/0'/0/2",
   'address': '0x573116f3a857707625Fe225c09AF9EaB1d1F1944',
   'privkey': '0x04d58976ef3d27ff991e2bc8759527b5299a08e19490a1a43a8597f214b62826',
   'pubkey': '03f610f63d257e169f16b9c6f68f406e01046c041759da85cfd2c98d1818918ba8'}],
 'btc-test': [{'path': "m/44'/1'/0'/0/0",
   'address': 'mqUWACnKFiA9WSxmwwxMLfWnDbwbvcMz6b',
   'privkey': 'cUmJ3ikWXUMQCYPRsmaCiia5ZxsxJK42X5UN6aQF5BiWjBb2SvRZ',
   'pubkey': '03f52019f42542dc4e31407fd67d09587452f

In [11]:
def priv_key_to_account(coin, priv_key):
    
    if coin == 'eth':
    
        return Account.from_key(priv_key)

    elif coin == "btc-test":
        
        return bit.PrivateKeyTestnet(priv_key)

In [14]:
eth_account = priv_key_to_account(ETH, '0x4c1deef92770ffb7850e34d5c2809dec86704773f3c1d4c33e6e103922a8a2bf')

In [15]:
btc_account = priv_key_to_account(BTCTEST, 'cW1aCxcauj29k7GwFBjTdKJwRJMiYJd6sYgoRMdeaAdkbYs12hT5')

In [64]:
def create_tx(coin, account, to, amount):
    
    if coin == 'eth':
        
        gasEstimate = w3.eth.estimateGas({'from': account.address, 
                       'to': to, 'value': amount})
        
        transaction = {'nonce': w3.eth.getTransactionCount(account.address), 
                       'gasPrice': w3.eth.gasPrice, 
                       'gas': gasEstimate,
                       'to':to, 
                       'from': account.address, 
                       'value':amount}
        
        return transaction
        
    elif coin == "btc-test":
        
        transaction = bit.PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])
        
        return transaction

In [56]:
to = '0x1F0dD3565bbC9D69d5656903782577525Bc48B6B'

In [58]:
to_btc = 'mjQedPF8y2hJtGNwC6Q7iTvqm2uJfzfdMD'

In [57]:
create_tx(ETH, eth_account, to, 100)

{'nonce': 0,
 'gasPrice': 20000000000,
 'gas': 21000,
 'to': '0x1F0dD3565bbC9D69d5656903782577525Bc48B6B',
 'from': '0xF7b703e1406D099be75FA815cCeb3c01dDd133f4',
 'value': 100}

In [ ]:
create_tx(BTCTEST, btc_account, to_btc, 2)

In [38]:
eth_account.address

'0xF7b703e1406D099be75FA815cCeb3c01dDd133f4'

In [42]:
gasestimate = w3.eth.estimateGas({'from': eth_account.address, 
                       'to': eth_account.address, 'value': 100})

In [43]:
transaction = {'nonce': w3.eth.getTransactionCount(eth_account.address), 
                       'gasPrice': w3.eth.gasPrice, 
                       'gas': gasestimate,
                       'to': to, 
                       'from': eth_account.address, 
                       'value':amount}

In [44]:
transaction

{'nonce': 0,
 'gasPrice': 20000000000,
 'gas': 21000,
 'to': '0x1F0dD3565bbC9D69d5656903782577525Bc48B6B',
 'from': '0xF7b703e1406D099be75FA815cCeb3c01dDd133f4',
 'value': 100}